In [ ]:
#@title Монтирование гугл диска
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.utils import capture

with capture.capture_output() as cap:
    def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
    print("[0;33mПодключаю...")
    drive.mount('/content/drive')
    mainpth="MyDrive"
clear_output()

inf('\u2714 Готово','success', '300px')

In [ ]:
#@title Установка репозитория (Запустить всего один раз за все использование)
from IPython.display import clear_output
import ipywidgets as widgets
from IPython.utils import capture

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

with capture.capture_output() as cap: 
    %cd /content/drive/$mainpth/
    !git clone https://github.com/NeuroDonu/SM.git
    !git pull
clear_output()

with capture.capture_output() as cap:
    %cd /content/drive/$mainpth/SM/assets/pretrained_models
    !wget https://huggingface.co/datasets/OwlMaster/gg2/resolve/main/inswapper_128.onnx
    !wget https://huggingface.co/NeuroDonu/faceparser/resolve/main/GFPGANv1.4.pth
    !wget https://huggingface.co/NeuroDonu/faceparser/resolve/main/79999_iter.pth
    !wget https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth
    !wget https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth
    !wget https://huggingface.co/ai-forever/Real-ESRGAN/resolve/main/RealESRGAN_x8.pth
    !wget https://huggingface.co/NeuroDonu/faceparser/resolve/main/det_10g.onnx
    !wget https://huggingface.co/bluefoxcreation/Codeformer-ONNX/resolve/main/codeformer.onnx
    !wget https://huggingface.co/NeuroDonu/faceparser/resolve/main/w600k_r50.onnx
clear_output()
inf('\u2714 Готово','success', '300px')

In [ ]:
#@title Установка модулей (Запускать при каждом использовании)
with capture.capture_output() as cap: 
    %cd /content/
    !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
    !mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
    !wget https://developer.download.nvidia.com/compute/cuda/12.2.2/local_installers/cuda-repo-ubuntu2204-12-2-local_12.2.2-535.104.05-1_amd64.deb
    !dpkg -i cuda-repo-ubuntu2204-12-2-local_12.2.2-535.104.05-1_amd64.deb
    !cp /var/cuda-repo-ubuntu2204-12-2-local/cuda-*-keyring.gpg /usr/share/keyrings/
    !apt-get update
    !apt-get -y install cuda
    !rm -r /content/cuda-repo-ubuntu2204-12-2-local_12.2.2-535.104.05-1_amd64.deb
clear_output()
with capture.capture_output() as cap:
    !apt-get install g++ gdb make ninja-build rsync zip
clear_output()
with capture.capture_output() as cap:
    %cd /content/drive/$mainpth/SM/
    %pip install torch==2.0.0+cpu torchvision==0.15.1+cpu torchaudio==2.0.1 --index-url https://download.pytorch.org/whl/cpu
    %pip install -r requirements.txt
clear_output()


In [ ]:
#@title Запуск
!python /content/drive/$mainpth/SM/app.py --colab --out_dir /content/results/